# BG-NBD ve Gamma-Gamma ile CLTV Tahmini

## İş Problemi
FLO satış ve pazarlama faaliyetleri için roadmap belirlemek istemektedir. Şirketin orta uzun vadeli plan yapabilmesi için var olan müşterilerin gelecekte şirkete sağlayacakları potansiyel değerin tahmin edilmesi gerekmektedir.

## Veri Seti Hikayesi
Veri seti Flo’dan son alışverişlerini 2020 - 2021 yıllarında OmniChannel (hem online hem offline alışveriş yapan) olarak yapan müşterilerin geçmiş alışveriş davranışlarından elde edilen bilgilerden oluşmaktadır.

### 📊 Veri Seti Değişkenleri

| Değişken Adı                         | Açıklama                                                                 |
|-------------------------------------|--------------------------------------------------------------------------|
| master_id                           | Eşsiz müşteri numarası                                                  |
| order_channel                       | Alışveriş yapılan platform (Android, iOS, Desktop, Mobile)             |
| last_order_channel                  | En son alışveriş yapılan kanal                                          |
| first_order_date                    | Müşterinin yaptığı ilk alışveriş tarihi                                |
| last_order_date                     | Müşterinin yaptığı son alışveriş tarihi                                |
| last_order_date_online              | Müşterinin online platformda yaptığı son alışveriş tarihi              |
| last_order_date_offline             | Müşterinin offline platformda yaptığı son alışveriş tarihi             |
| order_num_total_ever_online         | Online platformdaki toplam alışveriş sayısı                            |
| order_num_total_ever_offline        | Offline platformdaki toplam alışveriş sayısı                           |
| customer_value_total_ever_offline   | Offline alışverişlerde ödenen toplam ücret                             |
| customer_value_total_ever_online    | Online alışverişlerde ödenen toplam ücret                              |
| interested_in_categories_12         | Son 12 ayda alışveriş yapılan kategoriler                              |


## Ön Ayarlar

In [67]:
!pip install lifetimes

import datetime as dt
import pandas as pd
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter

In [68]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

## Veriyi Anlama ve Hazırlama (Data Understanding and Preparing)

In [69]:
# flo_data_20K.csv verisini okuyup Dataframe’in kopyasını oluşturma
df_ = pd.read_csv('/kaggle/input/flo-data-cltv/flo_data_20k.csv')
df = df_.copy()

In [70]:
#Veri setinde
# a. İlk 10 gözlem,
df.head(10)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0000,2.0000,159.9700,1853.5800,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0000,2.0000,189.9700,395.3500,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0000,1.0000,39.9900,81.9800,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0000,1.0000,49.9900,159.9900,[AKTIFSPOR]
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,Desktop,Offline,2018-11-18,2021-03-13,2018-11-18,2021-03-13,1.0000,2.0000,150.8700,49.9900,[KADIN]
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,Android App,Android App,2020-03-04,2020-10-18,2020-10-18,2020-03-04,3.0000,1.0000,59.9900,315.9400,[AKTIFSPOR]
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,Mobile,Offline,2020-05-15,2020-08-12,2020-05-15,2020-08-12,1.0000,1.0000,49.9900,113.6400,[COCUK]
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,Android App,Android App,2020-01-23,2021-03-07,2021-03-07,2020-01-25,3.0000,2.0000,120.4800,934.2100,"[ERKEK, COCUK, KADIN]"
9,1143f032-440d-11ea-8b43-000d3a38a36f,Mobile,Mobile,2019-07-30,2020-10-04,2020-10-04,2019-07-30,1.0000,1.0000,69.9800,95.9800,"[KADIN, AKTIFSPOR]"


In [71]:
# b. Değişken isimleri,
df.columns.to_list()

['master_id',
 'order_channel',
 'last_order_channel',
 'first_order_date',
 'last_order_date',
 'last_order_date_online',
 'last_order_date_offline',
 'order_num_total_ever_online',
 'order_num_total_ever_offline',
 'customer_value_total_ever_offline',
 'customer_value_total_ever_online',
 'interested_in_categories_12']

In [72]:
# c. Betimsel istatistik,
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.0000,3.1109,4.2256,1.0000,1.0000,2.0000,4.0000,200.0000
order_num_total_ever_offline,19945.0000,1.9139,2.0629,1.0000,1.0000,1.0000,2.0000,109.0000
customer_value_total_ever_offline,19945.0000,253.9226,301.5329,10.0000,99.9900,179.9800,319.9700,18119.1400
customer_value_total_ever_online,19945.0000,497.3217,832.6019,12.9900,149.9800,286.4600,578.4400,45220.1300


In [73]:
# d. Boş değer,
df.isnull().sum()

master_id                            0
order_channel                        0
last_order_channel                   0
first_order_date                     0
last_order_date                      0
last_order_date_online               0
last_order_date_offline              0
order_num_total_ever_online          0
order_num_total_ever_offline         0
customer_value_total_ever_offline    0
customer_value_total_ever_online     0
interested_in_categories_12          0
dtype: int64

In [74]:
# e. Değişken tipleri, incelemesi 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [75]:
# Aykırı değerleri baskılamak için gerekli olan outlier_thresholds ve replace_with_thresholds 
# fonksiyonlarının tanımlanması:
# Not: cltv hesaplanırken frequency değerleri integer olması gerekmektedir.
# Bu nedenle alt ve üst limitlerini round() ile yuvarlıyoruz.

def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = round(quartile3 + 1.5 * interquantile_range)
    low_limit = round(quartile1 - 1.5 * interquantile_range)
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    # dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [76]:
# "order_num_total_ever_online", "order_num_total_ever_offline", "customer_value_total_ever_offline",
# "customer_value_total_ever_online" değişkenlerinin varsa aykırı değerlerinin baskılanması

replace_with_thresholds(df, "order_num_total_ever_online")
replace_with_thresholds(df, "order_num_total_ever_offline")
replace_with_thresholds(df, "customer_value_total_ever_offline")
replace_with_thresholds(df, "customer_value_total_ever_online")

In [77]:
# Omnichannel müşterilerin hem online'dan hem de offline platformlardan alışveriş yaptığını 
# ifade etmektedir. Her bir müşterinin toplam alışveriş sayısı ve harcaması için yeni değişkenlerin oluşturulması

df["order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["customer_value_total"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

In [78]:
# Tarih ifade eden değişkenlerin tipinin date'e çevrilmesi

columns_to_convert_date = [col for col in df.columns if "date" in col]
df[columns_to_convert_date] = df[columns_to_convert_date].apply(pd.to_datetime)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

## CLTV Veri Yapısının Oluşturulması (Preparation of CLTV Data Structure)

In [79]:
# Veri setindeki en son alışverişin yapıldığı tarihten 2 gün sonrası analiz tarihi olarak alınmıştır
df["last_order_date"].max()
today_date = dt.datetime(2021, 6, 1)

In [80]:
# customer_id, recency_cltv_weekly, T_weekly, frequency ve monetary_cltv_avg değerlerinin yer aldığı 
# yeni bir cltv dataframe'inin oluşturulması. 
# NOT: Monetary değeri satın alma başına ortalama değer olarak, recency ve tenure değerleri ise 
# haftalık cinsten ifade edilecek.

cltv_df = pd.DataFrame()
cltv_df["customer_id"] = df["master_id"]
cltv_df["recency_cltv_weekly"] = (df["last_order_date"] - df["first_order_date"]).dt.days / 7
cltv_df["T_weekly"] = (today_date - df["first_order_date"]).dt.days / 7
cltv_df["frequency"] = df["order_num_total"]
cltv_df["monetary_cltv_avg"] = df["customer_value_total"] / df["order_num_total"]

cltv_df.describe().T

,count,mean,std,min,25%,50%,75%,max
recency_cltv_weekly,19945.0000,95.2635,74.5894,0.0000,50.4286,76.5714,109.4286,433.4286
T_weekly,19945.0000,114.4718,74.7710,0.7143,73.8571,93.0000,119.4286,437.1429
frequency,19945.0000,4.9781,4.1209,2.0000,3.0000,4.0000,6.0000,57.0000
monetary_cltv_avg,19945.0000,152.0990,73.5241,22.4900,103.6533,136.9062,182.4500,1401.8000


Yukarıdaki betimsel istatistik tablosunda görüldüğü üzere recency_cltv_weekly min değeri 0'dır. Bu sebeple BG/NBD modeli bize 'RuntimeWarning: invalid value encountered in sqrt' uyarısı verecektir. Aşağıdaki inceleme sonucunda recency_cltv_weekly değişkeninde 0'dan büyük olan değerlerin alınması için filtreleme yapılmıştır. 

In [81]:
(cltv_df["recency_cltv_weekly"] == 0).sum()

25

In [82]:
cltv_df = cltv_df[cltv_df["recency_cltv_weekly"] > 0]
(cltv_df["recency_cltv_weekly"] == 0).sum()

0

## BG/NBD, Gamma-Gamma Modellerinin Kurulması ve CLTV'nin hesaplanması

In [83]:
# BG/NBD modelinin kurulup fit edilmesi

bgf = BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(cltv_df["frequency"],
        cltv_df["recency_cltv_weekly"],
        cltv_df["T_weekly"])

<lifetimes.BetaGeoFitter: fitted with 19920 subjects, a: 0.00, alpha: 76.26, b: 0.00, r: 3.66>

In [84]:
# 3 ay içerisinde müşterilerden beklenen satın almaların tahmin edilmesi ve exp_sales_3_month olarak
# cltv dataframe'ine eklenmesi

cltv_df["exp_sales_3_month"] = bgf.predict(4 * 3,
                                           cltv_df["frequency"],
                                           cltv_df["recency_cltv_weekly"],
                                           cltv_df["T_weekly"])

In [85]:
# 6 ay içerisinde müşterilerden beklenen satın almaların tahmin edilmesi ve exp_sales_6_month olarak
# cltv dataframe'ine eklenmesi

cltv_df["exp_sales_6_month"] = bgf.predict(4 * 6,
                                           cltv_df["frequency"],
                                           cltv_df["recency_cltv_weekly"],
                                           cltv_df["T_weekly"])

In [86]:
# Gamma-Gamma modelinin kurulup fit edilmesi

ggf = GammaGammaFitter(penalizer_coef= 0.01)

ggf.fit(cltv_df["frequency"], cltv_df["monetary_cltv_avg"])

<lifetimes.GammaGammaFitter: fitted with 19920 subjects, p: 4.15, q: 0.47, v: 4.08>

In [87]:
# Müşterilerin ortalama bırakacakları değerin tahminlenip exp_average_value olarak cltv
# dataframe'ine eklenmesi

cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df["frequency"],
                                                                       cltv_df["monetary_cltv_avg"])

In [88]:
# 6 aylık CLTV hesaplanması ve cltv ismiyle dataframe'e eklenmesi
cltv_df["cltv"] = ggf.customer_lifetime_value(bgf,
                                              cltv_df["frequency"],
                                              cltv_df["recency_cltv_weekly"],
                                              cltv_df["T_weekly"],
                                              cltv_df["monetary_cltv_avg"],
                                              time=6, # aylık
                                              freq='W', # T'nin frekans bilgisi
                                              discount_rate=0.01)

In [89]:
# Cltv değeri en yüksek 20 kişinin gözlemlenmesi
cltv_df.sort_values(by="cltv", ascending=False)[:20]

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2.8571,7.8571,4.0000,1401.8000,1.0934,2.1869,1449.0490,3324.8728
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.1429,13.1429,11.0000,758.0855,1.9684,3.9367,767.3584,3169.5610
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51.7143,101.0000,7.0000,1106.4671,0.7220,1.4439,1127.6065,1708.3497
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,74.2857,74.5714,16.0000,501.8737,1.5645,3.1290,506.1657,1661.7573
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.7143,67.2857,52.0000,166.2246,4.6534,9.3068,166.7122,1627.9272
8868,9ce6e520-89b0-11ea-a6e7-000d3a38a36f,3.4286,34.4286,8.0000,601.2262,1.2646,2.5292,611.4902,1622.7087
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,8.2857,9.4286,2.0000,862.6900,0.7933,1.5866,923.6649,1537.6292
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.7143,13.0000,17.0000,259.8653,2.7781,5.5563,262.0724,1527.8188
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.5714,58.7143,31.0000,228.5300,3.0819,6.1638,229.6067,1484.9075
14858,031b2954-6d28-11eb-99c4-000d3a38a36f,14.8571,15.5714,3.0000,743.5867,0.8709,1.7418,778.0421,1421.9184


## CLTV Değerine Göre Segmentlerin Oluşturulması

In [90]:
# 6 aylık CLTV'ye göre tüm müşterilerinizi 4 gruba (segmente) ayrılması ve 
# grup isimlerinin veri setine eklenmesi

cltv_df["cltv_segment"] = pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])


In [91]:
# Veri setinin cltv değerlerine göre büyükten küçüğe sıralanması
cltv_df.sort_values(by="cltv", ascending=False)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv,cltv_segment
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2.8571,7.8571,4.0000,1401.8000,1.0934,2.1869,1449.0490,3324.8728,A
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.1429,13.1429,11.0000,758.0855,1.9684,3.9367,767.3584,3169.5610,A
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51.7143,101.0000,7.0000,1106.4671,0.7220,1.4439,1127.6065,1708.3497,A
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,74.2857,74.5714,16.0000,501.8737,1.5645,3.1290,506.1657,1661.7573,A
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.7143,67.2857,52.0000,166.2246,4.6534,9.3068,166.7122,1627.9272,A
...,...,...,...,...,...,...,...,...,...,...
2641,f7b3612e-a6ce-11e9-a2fc-000d3a38a36f,334.5714,361.2857,4.0000,34.9925,0.2102,0.4204,37.1986,16.4090,D
11232,f486e45e-a691-11e9-a2fc-000d3a38a36f,350.5714,378.1429,2.0000,45.2400,0.1496,0.2992,50.5011,15.8534,D
6963,7e0928c6-a6e3-11e9-a2fc-000d3a38a36f,342.0000,352.7143,4.0000,32.9350,0.2144,0.4288,35.0733,15.7806,D
27,c1f8f878-9f35-11e9-9897-000d3a38a36f,404.0000,417.1429,4.0000,32.7175,0.1864,0.3728,34.8487,13.6321,D
